# Monitoreo de sesgos del modelo con Amazon SageMaker: imparcialidad y explicabilidad con SageMaker Clarify

Amazon SageMaker Clarify ayuda a mejorar sus modelos de machine learning al detectar posibles sesgos y ayudar a explicar cómo hacen predicciones estos modelos. La funcionalidad de imparcialidad y explicabilidad brindada por SageMaker Clarify les permite a los clientes de AWS crear modelos de machine learning fiables y comprensibles. El producto incluye las herramientas necesarias para ayudarlo con las siguientes tareas:

- Medir los sesgos que pueden ocurrir durante cada etapa del ciclo de vida de machine learning (ML), como recopilación de datos, entrenamiento y ajuste de modelos y monitoreo de los modelos de ML implementados para inferencia.
- Generar informes de gobernanza del modelo para equipos de riesgo y cumplimiento, y reguladores externos.
- Brindar explicaciones de los datos, los modelos y los sistemas de monitoreo empleados para evaluar las predicciones.

## Importaciones

In [ ]:
import os
import copy
import json
import random
import time
import datetime
import boto3
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

### Configuración para esta tarea

In [ ]:
# Replace below <bucket_name> with the value the modelArtifactBucket value at the left of the instruction page
bucket = '<bucket_name>'

# Get production endpoint name
endpoint_name = boto3.Session().client('sagemaker').list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']

# Get production model name
model_name = boto3.Session().client('sagemaker').list_models(SortBy='CreationTime')['Models'][0]['ModelName']


print(f"Bucket Name :{bucket}")
print(f"Production Endpoint Name:{endpoint_name}")
print(f"Production Model Name:{model_name}")

In [ ]:
role = get_execution_role()
print(f"RoleArn: {role}")

sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

In [ ]:
print(f"Demo Bucket: {bucket}")
prefix = f'sagemaker/DEMO-ClarifyModelMonitor-{datetime.now():%Y%m%d%H%M%S}'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

code_prefix = '{}/code'.format(prefix)
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f'{s3_key}/datacapture'
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.xlarge"

## Archivos de datos

Lea los archivos de datos.

In [ ]:
test_dataset = "data/test.csv"
validation_dataset = "data/train.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

### Habilite la captura de datos en su punto de enlace

Para habilitar la captura de datos en tareas de monitoreo, especifique aquí la nueva opción de captura denominada DataCaptureConfig, la cual permite capturar la carga de solicitud y la carga de respuesta del punto de enlace. La configuración de captura se aplica a todas las variantes. 

Esta operación generalmente lleva de 10 a 15 minutos.

**Nota:** Esta tarea puede automatizarse mediante CloudFormation (revisor, sugerir opción)

In [ ]:
data_capture_config = DataCaptureConfig(
                        enable_capture = True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path,
                        kms_key_id=None,
                        capture_options=["REQUEST", "RESPONSE"],
                        csv_content_types=["text/csv"],
                        json_content_types=["application/json"])



# Now it is time to apply the new configuration and wait for it to be applied
predictor = Predictor(endpoint_name=endpoint_name)
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sagemaker_session.wait_for_endpoint(endpoint=endpoint_name)

### Realizar prueba del punto de enlace con tráfico de muestra

In [ ]:
# Here we are randomly picking subset of data from test datasets.
import itertools

shape = pd.read_csv(test_dataset)

a = [10*i for i in range(3)]
b = [10+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.drop(shape.columns[[0]],axis=1)
test_data = test_data.iloc[indices]
test_data_with_label = shape.iloc[indices]

In [ ]:
test_data.to_csv("data/data-test.csv",index=False,header=False)
test_data_with_label.to_csv("data/data-test-label.csv",index=False,header=False)

In [ ]:
# Invoke Endpoint with test data for a quick test
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
predictions = ''

with open('data/data-test.csv', 'r') as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip('\n')
        response = sagemaker_runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=payload)
        predictions = ','.join([predictions, response['Body'].read().decode('utf-8')])
        time.sleep(0.5)

predictions = predictions.replace('\n','')
predictions = predictions.split(",")
predictions.pop(0)
print("="*20)
print(predictions)
print("Done!") 

## Consultar los datos capturados

Ahora, debe enumerar los archivos de captura de datos almacenados en Amazon S3. Espere ver diferentes archivos para los diferentes períodos de tiempo, organizados según la hora en que ocurrió la invocación. El formato de la ruta de Amazon S3 es el siguiente:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
print(result)
str_error = "waiting for the file to load..."
while str_error:
    try:
        capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
        print(capture_files)
        str_error=None
    except:
        print(str_error)
        time.sleep(10)
        contine
print("Found Capture Files:")
print("\n ".join(capture_files))

Luego, observe el contenido de un único archivo de captura. Aquí, debería ver todos los datos capturados en un archivo con formato de línea de JSON específico de Amazon SageMaker. Eche un vistazo a las primeras líneas del archivo de captura.

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

Por último, le presentamos a continuación el contenido de una única linea de un archivo con formato de línea de JSON para que pueda verlo mejor.

In [ ]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

### Configuración de Amazon SageMaker Clarify

**Nota:** Las muestras de código de abajo se tomaron de [código de muestra] (https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb), brindado por Amazon SageMaker. Algunos de los códigos se actualizan según los requisitos del laboratorio.

In [ ]:
# Run SageMaker Clarify processor using the following command
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

Cargue los conjuntos de datos de prueba y entrenamiento a S3. Se usarán los términos _train_uri_ y _test_uri_ para obtener datos directamente de la escritura de dataconfig y modelconfig de S3 en los siguientes pasos.

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('data/train.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
test_uri = S3Uploader.upload('data/test.csv', 's3://{}/{}'.format(bucket, prefix))

## Detección de sesgos

SageMaker Clarify lo ayuda a detectar posibles sesgos antes y después del entrenamiento usando diferentes métricas.

## Escribir dataconfig y modelconfig

Un objeto **DataConfig** comunica información básica sobre la E/S de datos en SageMaker Clarify. Usted debe especificar dónde se puede encontrar el conjunto de datos de entrada, dónde se debe almacenar la salida de datos, cuál es la columna objetivo (etiqueta), cuáles son los nombres de los encabezados y cuál es el tipo de conjunto de datos.

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=all_headers,
                                      dataset_type='text/csv')

Un objeto **ModelConfig** comunica información sobre su modelo entrenado. Para evitar que haya tráfico adicional hacia sus modelos de producción, SageMaker Clarify crea y luego desarma un punto de enlace especializado al realizar el procesamiento.

- **instance_type** e **instance_count** especifican su tipo de instancia y su conteo de instancia preferidos para ejecutar su modelo durante el procesamiento de SageMaker Clarify. El conjunto de datos de la prueba es pequeño, por lo cual una sola instancia estándar es suficiente para ejecutar este ejemplo. Si tiene un conjunto de datos complejo y grande, le convendría usar un mejor tipo de instancia para acelerar el proceso o agregar más instancias para permitir la paralelización de Spark.

- **accept_type** se refiere al formato de la carga de respuesta del punto de enlace y **content_type** se refiere al formato de la carga de solicitud del punto del enlace.

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

El objeto **ModelPredictedLabelConfig** brinda información sobre el formato de sus predicciones. El modelo de bosque aleatorio genera probabilidades de muestras, por lo cual SageMaker Clarify invoca al punto de enlace y luego usa el término probability_threshold para convertir la probabilidad a etiquetas binarias para el análisis de sesgos. La predicción que esté por encima del umbral se interpretará como valor de etiqueta 1, y la que sea igual o esté por debajo del umbral, como valor de etiqueta 0.

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

### Escribir BiasConfig
SageMaker Clarify también necesita información sobre cuáles son las columnas sensibles (facets), cuáles pueden ser las características sensibles (facet_values_or_threshold) y cuáles son los resultados deseados (label_values_or_threshold). SageMaker Clarify puede manejar datos categóricos y continuos para facet_values_or_threshold y para label_values_or_threshold. En este caso, estamos usando datos categóricos.

Especificamos esta información en la API de BiasConfig. Aquí, donde el resultado positivo es Churn = 0, _Account Length_ es una categoría sensible y los respondedores de_Day Calls_ son el grupo sensible. group_name se usa para crear subgrupos para la medición.

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='Account Length',
                                facet_values_or_threshold=[100],
                                group_name='Day Calls')

**Sesgo anterior al entrenamiento**

El sesgo puede presentarse en los datos antes de que se realice el entrenamiento de los modelos. Inspeccionar los datos para buscar sesgos antes del entrenamiento puede ayudarlo a detectar brechas en la recolección de datos, informar sus decisiones de ingeniería de características y comprender qué sesgos sociales pueden reflejar sus datos.

El cómputo de las métricas de sesgos antes del entrenamiento no requiere un modelo entrenado.

**Sesgo posterior al entrenamiento**

El cómputo de las métricas de sesgos después del entrenamiento sí requiere un modelo entrenado.

Los datos entrenados sin sesgos (según lo determinen conceptos de imparcialidad medidos por métricas de sesgo) pueden provocar predicciones del modelo con sesgos después del entrenamiento. Que esto ocurra o no dependerá de varios factores, incluidas las elecciones de hiperparámetros.

Puede ejecutar estas opciones por separado con run_pre_training_bias() y run_post_training_bias() o al mismo tiempo con run_bias(), como se muestra abajo.

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

**Ver el informe de sesgos**

En Studio, puede ver los resultados debajo de la pestaña Experiments (Experimentos).

<img src="./recordings/bias_report.gif">

Cada métrica de sesgo tiene una explicación detallada con ejemplos que puede explorar.

<img src="./recordings/bias_detail.gif">

También puede resumir los resultados en una tabla práctica.

<img src="./recordings/bias_report_chart.gif">



Si no usa Studio aún, puede acceder al informe de sesgos en formato pdf, html e ipynb en el siguiente bucket de S3:

In [ ]:
bias_report_output_path

### Explicación de las predicciones

Existen cada vez más necesidades comerciales y reglamentaciones legislativas que requieren que se indique por qué un modelo tomó una decisión. SageMaker Clarify usa el algoritmo SHAP para explicar la contribución que hace cada característica de entrada a la decisión final.

El algoritmo Kernel SHAP requiere una línea de referencia (también llamado “conjunto de datos de fondo”). Los conjuntos de datos de línea de referencia deberán ser los mismos que dataset_type de DataConfig, y las muestras de línea de base solo deberán incluir características. Por definición, la línea de referencia debería ser una URI de S3 al archivo de conjunto de datos de línea de referencia o una lista de muestras en el lugar. En este caso, elegimos la segunda opción e incluimos la primera muestra del conjunto de datos de prueba en la lista.

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=all_headers,
                                dataset_type='text/csv')

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

#### Ver el informe de explicabilidad
Al igual que con el informe de sesgos, puede ver el informe de explicabilidad en Studio en la pestaña Experiments (Experimentos).


<img src="./recordings/explainability_detail.gif">

La pestaña Model Insights (Información del modelo) incluye vínculos directos al informe y a la información del modelo.

Si no usa Studio aún, al igual que con el informe de sesgos, puede acceder este informe en el siguiente bucket de S3:

In [ ]:
explainability_output_path

### Limpieza

Por último, recuerde limpiar los recursos que configuramos y usamos para esta demostración.

In [ ]:
session.delete_model(model_name)